In [11]:
import pandas as pd
from pathlib import Path


In [12]:
csv_path = Path("Resource/Data1.csv")
df = pd.read_csv(csv_path)
df.head(50)

C:\Users\Randg\AppData\Local\Temp\ipykernel_33516\1847774805.py:2: DtypeWarning: Columns (13) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(csv_path)


,REF_DATE,GEO,DGUID,Prices,Seasonal adjustment,Estimates,UOM,UOM_ID,SCALAR_FACTOR,SCALAR_ID,VECTOR,COORDINATE,VALUE,STATUS,SYMBOL,TERMINATED,DECIMALS
0,1961-01,Canada,2016A000011124,Chained (2012) dollars,Seasonally adjusted at annual rates,Food and non-alcoholic beverages [C11],Dollars,81,millions,6,v61988958,1.1.1.1,37146.0,NaN,NaN,NaN,0
1,1961-01,Canada,2016A000011124,Chained (2012) dollars,Seasonally adjusted at annual rates,Food [C111],Dollars,81,millions,6,v61988959,1.1.1.2,34480.0,NaN,NaN,NaN,0
2,1961-01,Canada,2016A000011124,Chained (2012) dollars,Seasonally adjusted at annual rates,Non-alcoholic beverages [C112],Dollars,81,millions,6,v61988960,1.1.1.3,2960.0,NaN,NaN,NaN,0
3,1961-01,Canada,2016A000011124,Chained (2012) dollars,Seasonally adjusted at annual rates,"Alcoholic beverages, tobacco and cannabis [C12]",Dollars,81,millions,6,v61988961,1.1.1.4,14616.0,NaN,NaN,NaN,0
4,1961-01,Canada,2016A000011124,Chained (2012) dollars,Seasonally adjusted at annual rates,Alcoholic beverages [C121],Dollars,81,millions,6,v61988962,1.1.1.5,5198.0,NaN,NaN,NaN,0
5,1961-01,Canada,2016A000011124,Chained (2012) dollars,Seasonally adjusted at annual rates,Tobacco [C122],Dollars,81,millions,6,v61988963,1.1.1.6,17282.0,NaN,NaN,NaN,0
6,1961-01,Canada,2016A000011124,Chained (2012) dollars,Seasonally adjusted at annual rates,Cannabis [C123],Dollars,81,millions,6,v1043024085,1.1.1.61,372.0,NaN,NaN,NaN,0
7,1961-01,Canada,2016A000011124,Chained (2012) dollars,Seasonally adjusted at annual rates,Clothing and footwear [C13],Dollars,81,millions,6,v61988964,1.1.1.7,7931.0,NaN,NaN,NaN,0
8,1961-01,Canada,2016A000011124,Chained (2012) dollars,Seasonally adjusted at annual rates,Clothing [C131],Dollars,81,millions,6,v61988965,1.1.1.8,6432.0,NaN,NaN,NaN,0
9,1961-01,Canada,2016A000011124,Chained (2012) dollars,Seasonally adjusted at annual rates,Footwear [C132],Dollars,81,millions,6,v61988966,1.1.1.9,1595.0,NaN,NaN,NaN,0


In [14]:
# Assume df is your DataFrame and 'date' is your column with dates
df['REF_DATE'] = pd.to_datetime(df['REF_DATE'], format='%Y-%m')

# Set 'date' as index if it's not yet
df.set_index('REF_DATE', inplace=True)

# Resample to yearly data, taking the mean of each year
df_yearly = df.resample('Y').mean()


df_yearly

C:\Users\Randg\AppData\Local\Temp\ipykernel_33516\2952654253.py:8: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  df_yearly = df.resample('Y').mean()


,UOM_ID,SCALAR_ID,VALUE,SYMBOL,DECIMALS
REF_DATE,,,,,
1961-12-31,112.600000,4.800000,6612.577399,NaN,0.600000
1962-12-31,112.703947,4.796053,6576.424446,NaN,0.601974
1963-12-31,112.703947,4.796053,6846.216622,NaN,0.601974
1964-12-31,112.703947,4.796053,7202.492289,NaN,0.601974
1965-12-31,112.703947,4.796053,7608.543418,NaN,0.601974
...,...,...,...,...,...
2019-12-31,120.500000,4.500000,51183.022185,NaN,0.750000
2020-12-31,120.500000,4.500000,48454.001948,NaN,0.750000
2021-12-31,120.500000,4.500000,51846.293490,NaN,0.750000


In [18]:
clean_df = df_yearly[["VALUE"]]

clean_df

,VALUE
REF_DATE,
1961-12-31,6612.577399
1962-12-31,6576.424446
1963-12-31,6846.216622
1964-12-31,7202.492289
1965-12-31,7608.543418
...,...
2019-12-31,51183.022185
2020-12-31,48454.001948
2021-12-31,51846.293490


SyntaxError: invalid syntax (3121196849.py, line 1)